In [1]:
import numpy as np
import pandas as pd

pd.set_option('precision', 2)

In [2]:
sp5 = pd.read_parquet('sp5.parq')\
    .set_index(['date', 'ticker'])\
    .sort_index()
sp5.head()

,,adj_close
date,ticker,
2015-01-02,SP500,2058.20
2015-01-05,SP500,2020.58
2015-01-06,SP500,2002.61
2015-01-07,SP500,2025.90
2015-01-08,SP500,2062.14


In [3]:
tbill = pd.read_csv('tbill-2015.csv', parse_dates=['date'])

In [7]:
tbill.head()

,rate4week,rate52week
date,,
2015-01-02,0.02,0.22
2015-01-05,0.02,0.23
2015-01-06,0.02,0.25
2015-01-07,0.02,0.25
2015-01-08,0.01,0.23


In [4]:
tbill.dtypes

date          datetime64[ns]
rate4week            float64
rate52week           float64
dtype: object

In [5]:
tbill.set_index('date', inplace=True)

In [6]:
# check that the index is valid
tbill.index.is_monotonic, tbill.index.is_unique

(True, True)

In [8]:
# set these to be the first day of the month
tbill_mnth = tbill.resample('MS').first()

In [9]:
tbill_mnth

,rate4week,rate52week
date,,
2015-01-01,0.02,0.22
2015-02-01,0.01,0.15
2015-03-01,0.02,0.19
2015-04-01,0.02,0.26
2015-05-01,0.00,0.24
2015-06-01,0.02,0.25
2015-07-01,0.01,0.27
2015-08-01,0.02,0.31
2015-09-01,0.01,0.37


In [10]:
# incomplete overlap of indexes
tbill_mnth.index & sp5.index.get_level_values('date')

DatetimeIndex(['2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01',
               '2015-09-01', '2015-10-01', '2015-12-01'],
              dtype='datetime64[ns]', name='date', freq=None)

In [11]:
# try reindexing against MultiIndex
try:
    tbill_mnth.reindex(sp5.index, level='date', method='ffill')
except TypeError:
    print("'level' and 'method' cannot be used simultaneously")

'level' and 'method' cannot be used simultaneously


In [12]:
tbill_daily = tbill_mnth.reindex(
    sp5.index.get_level_values('date'), 
    method='ffill')

In [13]:
# see that the forward fill worked
tbill_daily['2015-01'][:3]

,rate4week,rate52week
date,,
2015-01-02,0.02,0.22
2015-01-05,0.02,0.22
2015-01-06,0.02,0.22


In [14]:
tbill_daily['2015-02'][:3]

,rate4week,rate52week
date,,
2015-02-02,0.01,0.15
2015-02-03,0.01,0.15
2015-02-04,0.01,0.15


In [15]:
sp5.join(tbill_daily, on='date')

,,adj_close,rate4week,rate52week
date,ticker,,,
2015-01-02,SP500,2058.20,0.02,0.22
2015-01-05,SP500,2020.58,0.02,0.22
2015-01-06,SP500,2002.61,0.02,0.22
2015-01-07,SP500,2025.90,0.02,0.22
2015-01-08,SP500,2062.14,0.02,0.22
...,...,...,...,...
2015-12-24,SP500,2060.99,0.19,0.48
2015-12-28,SP500,2056.50,0.19,0.48
2015-12-29,SP500,2078.36,0.19,0.48
